# 🚀 Quick Start - Panel Dashboard

**最新版本**: Phase 2.0 - Panel Integration  
**特性**: 可导出静态 HTML（控件 + 图表都可交互）

---

## 📋 本 Notebook 的步骤

1. **初始化环境** - 加载框架 + CSS 宽度修复
2. **加载数据** - 从 Parquet 文件加载
3. **生成基本 AI Context** - 数据结构信息
4. **生成完整 AI Prompt** - 包含所有文档（复制给 AI）
5. **选择维度** - 选择分析维度
6. **创建 Panel 仪表盘** - AI 生成分析逻辑
7. **导出 HTML** - 分享给他人

---


## 📦 Step 1: 初始化环境

加载框架 + CSS 宽度修复（让图表占满屏幕）

In [1]:
# ========================================
# 移除 JupyterLab 宽度限制（关键！）
# ========================================
from IPython.display import HTML, display

display(HTML("""
<style>
    /* 核心：解除 JupyterLab 4 的宽度限制 */
    .jp-Notebook { 
        --jp-notebook-max-width: 100% !important; 
    }
    
    /* 确保所有输出容器占满宽度 */
    .jp-Notebook-cell, 
    .jp-Cell-outputWrapper, 
    .jp-OutputArea-output, 
    .jp-OutputArea-child { 
        max-width: none !important; 
        width: 100% !important; 
    }
    
    /* Panel 根容器强制铺满 */
    .bk-root, .bk-root > .bk { 
        width: 100% !important; 
        max-width: none !important; 
    }
</style>
"""))

print("✅ JupyterLab 宽度限制已移除")

# ========================================
# 加载框架
# ========================================
import polars as pl
import plotly.express as px
import panel as pn

from src.session import DataSession
from src.dashboard import PanelDashboardBuilder
from src.utils import print_markdown_table

# 初始化 Panel（全局设置 stretch_width）
pn.extension('plotly', sizing_mode='stretch_width')

print("✅ 环境初始化完成")
print("📚 框架版本: Phase 2.0 - Panel Integration")


✅ JupyterLab 宽度限制已移除


✅ 环境初始化完成
📚 框架版本: Phase 2.0 - Panel Integration


## 📂 Step 2: 加载数据

从 Parquet 文件加载数据

In [2]:
# 创建数据会话
session = DataSession()

# 加载数据（替换为你的数据文件）
session.load("insurance_data_cleaned", alias="df")  # 或 session.load("your_data.parquet", alias="df")

# 验证数据
print(f"✅ 数据已加载: {df_df.height:,} 行 × {df_df.width} 列")
print(f"📊 数据预览:\n")
print_markdown_table(df_df.head(5))


✅ 已加载: df_df (insurance_data_cleaned)
✅ 数据已加载: 776,739 行 × 69 列
📊 数据预览:



|   业务年度 |   机构代码 | 机构名称                         |   业务险种 |                 保单号 |   危险单位号 | 支付币别   | 业务来源   | 是否联共保   |   联共保我司占比 |         地址代码 | 标的地址或船名                                                   | 被保险人名称               | 保险起期   | 保险止期   |   风险代码 | 风险代码描述                                         |   风险等级 |      总保额 |   总保费 |   总保费_原币 |   手续费 |   总已决赔款 |   总未决赔款 |    自留保额 |   自留保费 |   自留已决 |   自留未决 |   附加自留保额 |   附加自留保费 |   附加自留已决 |   附加自留未决 |   协议分保额 |   协议分保费 |   协议摊回费用 |   协议已决 |   协议未决 |   成数分保额 |   成数分保费 |   成数摊回费用 |   成数已决 |   成数未决 |   溢额分保额 |   溢额分保费 |   溢额摊回费用 |   溢额已决 |   溢额未决 |   OPENCOVER分保额 |   OPENCOVER分保费 |   OPENCOVER摊回费用 |   OPENCOVER已决 |   OPENCOVER未决 |   临分分保额 |   临分分保费 |   临分摊回费用 |   临分已决 |   临分未决 | 占用性质名称   | 行业类别名称   | 境内境外   |   分入手续费率 |   分入手续费 |   分入出单费比例 |   分入出单费 |   分入经纪费率 |   分入经纪费 | 分入经纪人名称   | 直保代理人名称                           | 归属机构   |
|-----------:|-----------:|:---------------------------------|-----------:|-----------------------:|-------------:|:-----------|:-----------|:-------------|-----------------:|-----------------:|:-----------------------------------------------------------------|:---------------------------|:-----------|:-----------|-----------:|:-----------------------------------------------------|-----------:|------------:|---------:|--------------:|---------:|-------------:|-------------:|------------:|-----------:|-----------:|-----------:|---------------:|---------------:|---------------:|---------------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|------------------:|------------------:|--------------------:|----------------:|----------------:|-------------:|-------------:|---------------:|-----------:|-----------:|:---------------|:---------------|:-----------|---------------:|-------------:|-----------------:|-------------:|---------------:|-------------:|:-----------------|:-----------------------------------------|:-----------|
|       2024 |   44030033 | 深圳分公司工商保险业务部         |       0102 | 3010220244403000000577 |            1 | CNY        | 直接业务   | 非联共保     |            100   | 4403003300008323 | 广东省-深圳市-福田区滨河大道与益田路交界东南皇庭世纪花园1号楼14A | 崔鹏                       | 2024-04-23 | 2027-07-22 |      24107 | 中行渠道财产险类一般行业                             |          1 | 7.8329e+06  |  1921.27 |       1921.27 |     0    |            0 |            0 | 7.8329e+06  |    1921.27 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 综合商业类     |                | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  |                                          | 深圳       |
|       2016 |   37070004 | 山东分公司潍坊中心支公司银保团队 |       0101 | 3010120163707000000006 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3707000300000714 | 山东省潍坊市滨海经济开发区新海大街99号                           | 潍坊友容实业有限公司       | 2016-01-06 | 2017-01-05 |      11160 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 3.97982e+07 | 27858.8  |      27858.8  |  4178.81 |            0 |            0 | 3.97982e+07 |   27858.8  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 建筑安装业     | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司潍坊分行             | 山东       |
|       2023 |   31030001 | 上海自贸试验区分公司经理室       |       0104 | 3010420233103000001350 |            1 | CNY        | 经纪业务   | 从共         |             10.5 | 3100001700042357 | 新疆维吾尔自治区哈密地区哈密市伊州区                             | 中国广核集团               | 2023-02-01 | 2023-09-30 |      23056 | 市场渠道共保财产险类电厂及水电站（不含联合循环电厂） |          2 | 1.33116e+08 | 21421.9  |      21421.9  |  3406.08 |            0 |            0 | 1.2e+08     |   19311.2  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  1.31159e+07 |      2110.69 |         675.42 |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 二级工业       | 其他电力生产   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中广核保险经纪有限责任公司               | 上海       |
|       2020 |   33070007 | 浙江分公司金华中心支公司银保团队 |       0101 | 3010120203307000000114 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3307000800003824 | 浙江省金华市兰溪市兰江街道毕家村                                 | 兰溪市三江仓储发展有限公司 | 2020-05-23 | 2021-05-22 |      20107 | 中行渠道财产险类一般行业                             |          3 | 1.23e+06    |  1160.38 |       1160.38 |     0    |            0 |            0 | 1.23e+06    |    1160.38 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 工业           | 内河货物运输   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司兰溪支行             | 浙江       |
|       2011 |   32010004 | 江苏分公司南京营业部银保团队     |       0102 |  301022010320100000012 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3200001000000077 | 江苏省南京市太平北路106号第四至七层                              | 宏图三胞高科技术有限公司   | 2011-01-01 | 2011-12-31 |      10383 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 2.95e+07    | 14750    |      14750    |  2360    |            0 |            0 | 2.832e+07   |   14160    |          0 |          0 |              0 |              0 |              0 |              0 |     1.18e+06 |          590 |          241.9 |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 其他科技服务   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行南京地区鼓楼支行公司板块公司业务 | 江苏       |

## 🤖 Step 3: 生成基本 AI Context

生成数据概览

In [3]:
# 生成基本 AI Context
print("📋 数据结构信息:\n")
print("=" * 80)
print(session.get_ai_context())
print("=" * 80)

print("\n💡 提示: 下一步会生成完整的 AI Prompt")


📋 数据结构信息:

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float64)
- `溢额未决` (Float64

## 📋 Step 4: 生成完整 AI Prompt

**运行这个 cell，然后复制全部输出给 AI**

这个 cell 会输出：
- 数据结构信息
- 完整的 Panel Dashboard 使用指南
- 代码模板
- 常见错误对比
- 检查清单


In [4]:
# ========================================
# 📋 给 AI 的完整信息（复制全部输出给 AI）
# ========================================

# 直接执行最新的 Step 4 模板
import os

# 自动找到项目根目录
def find_project_root():
    """向上查找包含 src/ 和 notebooks/templates/ 的项目根目录"""
    current = os.path.abspath('.')
    while current != '/':
        # 检查是否存在项目标志
        if (os.path.exists(os.path.join(current, 'src')) and 
            os.path.exists(os.path.join(current, 'notebooks', 'templates'))):
            return current
        # 向上一级
        current = os.path.dirname(current)
    # 找不到就返回当前目录
    return os.path.abspath('.')

project_root = find_project_root()
step4_path = os.path.join(project_root, 'notebooks', 'templates', 'step4_standalone.py')

# 调试信息（可选，帮助诊断）
# print(f"项目根目录: {project_root}")
# print(f"Step 4 路径: {step4_path}")
# print(f"文件存在: {os.path.exists(step4_path)}")

if os.path.exists(step4_path):
    exec(open(step4_path).read())
else:
    # 如果文件找不到，使用嵌入版本
    print("=" * 80)
    print("📋 **复制以下所有内容给 AI**")
    print("=" * 80)
    print()
    print("## 📊 数据结构")
    print()
    print(session.get_ai_context())
    print()
    print("=" * 80)
    print("## 📚 Panel Dashboard 完整使用指南")
    print("=" * 80)
    print()
    print("⚠️ 规则 0: 禁止硬编码任何维度！")
    print("必须使用: group_col = values.get('_aggregation_dimension')")
    print("必须跳过: if dim == '_aggregation_dimension': continue")
    print()
    print("详细说明见: notebooks/templates/step4_standalone.py")
    print("=" * 80)


📋 **复制以下所有内容给 AI**

## 📊 数据结构

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float6

## 🎯 Step 5: 选择分析维度

选择你想要分析的维度字段

In [5]:
# 👤 选择你要分析的维度
# 修改下面的列表，添加或删除维度

selected_dimensions = [
    '业务年度',
    '业务险种',
    '归属机构'  # 取消注释以启用
    # '境内境外',  # 取消注释以启用
]

print(f"✅ 已选择 {len(selected_dimensions)} 个维度:")
for dim in selected_dimensions:
    if dim in df_df.columns:
        n_unique = df_df[dim].n_unique()
        print(f"  - {dim} ({n_unique:,} 个唯一值)")
    else:
        print(f"  ⚠️  {dim} - 不存在于数据中")

print("\n💡 下一步: 创建 Panel 仪表盘")


✅ 已选择 3 个维度:
  - 业务年度 (25 个唯一值)
  - 业务险种 (26 个唯一值)
  - 归属机构 (30 个唯一值)

💡 下一步: 创建 Panel 仪表盘


## 📊 Step 6: 创建 Panel 仪表盘

自动创建仪表盘控件，然后让 AI 生成分析逻辑

In [6]:
# 创建 Panel 仪表盘（自动生成控件）
dashboard = PanelDashboardBuilder.from_data(
    df_df,
    dimensions=selected_dimensions,
    title="📊 数据分析仪表盘"
)

print("\n💡 下一步: 让 AI 生成分析逻辑")
print("\n📝 给 AI 的提示:")
print("="  * 80)
print("请使用 Panel Dashboard 生成分析代码。")
print(f"已选择维度: {selected_dimensions}")
print("需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]")
print("="  * 80)


🎨 从数据创建仪表盘: 📊 数据分析仪表盘
📊 数据维度: 776,739 行 × 69 列
🔧 配置维度字段: 业务年度, 业务险种, 归属机构



  ✅ 业务年度: MultiChoice (25 个选项 + 全选, 默认选 1 个)
  ✅ 业务险种: MultiChoice (26 个选项 + 全选, 默认选 1 个)
  ✅ 归属机构: MultiChoice (30 个选项 + 全选, 默认选 1 个)

  ✅ 聚合维度选择器: Select (3 个维度可选, 默认: 业务年度)

✅ 仪表盘控件创建完成 (4 个控件)
💡 下一步: 使用 dashboard.set_update_function(your_function)

📄 等效代码（可复制用于自定义）:

```python
import panel as pn
import polars as pl

dashboard = PanelDashboardBuilder(title="📊 数据分析仪表盘")

# 示例: 业务年度
widget = pn.widgets.Select(name='业务年度', options=[...])
dashboard.widgets['业务年度'] = widget

# 示例: 业务险种
widget = pn.widgets.Select(name='业务险种', options=[...])
dashboard.widgets['业务险种'] = widget

# 定义更新函数
@pn.depends(*dashboard.widgets.values())
def update(*args):
    # 你的分析逻辑
    return fig

dashboard.set_update_function(update)
dashboard.show()  # Jupyter 显示
dashboard.save('output.html')  # 导出 HTML
```



💡 下一步: 让 AI 生成分析逻辑

📝 给 AI 的提示:
请使用 Panel Dashboard 生成分析代码。
已选择维度: ['业务年度', '业务险种', '归属机构']
需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]


## 🤖 Step 7: AI 生成分析逻辑

**在这里粘贴 AI 生成的代码**

AI 应该生成：
1. `@pn.depends` 装饰的更新函数
2. 数据过滤和聚合逻辑
3. Plotly 图表
4. `dashboard.show()` 和 `dashboard.save()`


In [7]:
import panel as pn
import polars as pl
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.utils import print_markdown_table

# ==============================================================================
# 最终纯净版：无人工缩放 + 2005年后 + 完美布局
# ==============================================================================
@pn.depends(*dashboard.widgets.values())
def update_dashboard(*args):
    # --- 1. 获取控件 ---
    values = {name: w.value for name, w in dashboard.widgets.items()}
    group_col = values.get('_aggregation_dimension', '业务险种')
    
    # --- 2. 动态过滤 ---
    filters = []
    
    # 过滤掉 2005 年以前的数据
    try:
        filters.append(pl.col('业务年度').cast(pl.Int64) >= 2005)
    except:
        pass 
    
    for dim, val in values.items():
        if dim == '_aggregation_dimension': continue
        if isinstance(val, list):
            if '全选' not in val: filters.append(pl.col(dim).is_in(val))
        elif val != '全选':
            filters.append(pl.col(dim) == val)
            
    filtered = df_df
    for f in filters: filtered = filtered.filter(f)
    
    if filtered.height == 0: return f"⚠️ 按【{group_col}】筛选无数据 (仅显示 2005 及以后)"

    # --- 3. 聚合计算 ---
    reins_types = {'协议': '协议','成数': '成数','溢额': '溢额','Open': 'OPENCOVER','临分': '临分'}

    agg_exprs = [
        # 基础数据
        pl.col('总保费').sum(),
        pl.col('自留保费').sum(),
        (pl.col('总已决赔款') + pl.col('总未决赔款')).sum().alias('毛赔款'),
        (pl.col('自留已决') + pl.col('自留未决')).sum().alias('自留赔款'),
        pl.col('手续费').sum().alias('直保手续费'),
        (pl.col('分入手续费').fill_null(0) + pl.col('分入出单费').fill_null(0) + pl.col('分入经纪费').fill_null(0)).sum().alias('总分入费用'),
        (pl.col('协议摊回费用').fill_null(0) + pl.col('成数摊回费用').fill_null(0) + pl.col('溢额摊回费用').fill_null(0) + 
         pl.col('OPENCOVER摊回费用').fill_null(0) + pl.col('临分摊回费用').fill_null(0)).sum().alias('总摊回费用')
    ]

    for name, prefix in reins_types.items():
        agg_exprs.append(pl.col(f'{prefix}分保费').fill_null(0).sum().alias(f'{name}_保费'))
        agg_exprs.append((pl.col(f'{prefix}已决').fill_null(0) + pl.col(f'{prefix}未决').fill_null(0)).sum().alias(f'{name}_赔款'))
        agg_exprs.append(pl.col(f'{prefix}摊回费用').fill_null(0).sum().alias(f'{name}_摊回'))

    result = filtered.group_by(group_col).agg(agg_exprs)
    
    # --- 4. 衍生指标计算 ---
    df_plot = result.to_pandas()
    div = 10000 
    
    # [A] 成本计算 (单位：元)
    df_plot['总获取成本'] = df_plot['直保手续费'] + df_plot['总分入费用']
    df_plot['毛成本'] = df_plot['毛赔款'] + df_plot['总获取成本']
    df_plot['自留成本'] = df_plot['自留赔款'] + df_plot['总获取成本'] - df_plot['总摊回费用']
    
    df_plot['分出保费'] = df_plot['总保费'] - df_plot['自留保费']
    df_plot['分出赔款'] = df_plot['毛赔款'] - df_plot['自留赔款']
    
    # [B] 核心比率计算 (元/元) - 确保逻辑正确
    def calc_ratio(num, den):
        return df_plot.apply(lambda x: (x[num]/x[den]*100) if x[den]!=0 else 0, axis=1)

    df_plot['分出比例'] = calc_ratio('分出保费', '总保费')
    df_plot['毛综合成本率'] = calc_ratio('毛成本', '总保费')
    df_plot['自留综合成本率'] = calc_ratio('自留成本', '自留保费') 
    df_plot['总分出综合成本率'] = df_plot.apply(lambda x: ((x['分出赔款']+x['总摊回费用'])/x['分出保费']*100) if x['分出保费']!=0 else 0, axis=1)

    # [C] 万元转换 (仅用于展示)
    df_plot['毛保费_万'] = df_plot['总保费'] / div
    df_plot['自留保费_万'] = df_plot['自留保费'] / div
    df_plot['自留成本_万'] = df_plot['自留成本'] / div
    df_plot['自留利润_万'] = (df_plot['自留保费'] - df_plot['自留成本']) / div
    
    for name in reins_types.keys():
        df_plot[f'{name}_保费_万'] = df_plot[f'{name}_保费'] / div
        df_plot[f'{name}_利润_万'] = (df_plot[f'{name}_保费'] - (df_plot[f'{name}_赔款'] + df_plot[f'{name}_摊回'])) / div
        
        df_plot[f'{name}_COR'] = df_plot.apply(
            lambda x: ((x[f'{name}_赔款'] + x[f'{name}_摊回']) / x[f'{name}_保费'] * 100) 
            if x[f'{name}_保费'] != 0 else 0, axis=1
        )

    # 排序
    df_plot = df_plot.sort_values(group_col, ascending=True)

    # ⚡️ 移除智能可视范围代码 (已删除)

    # --- 5. 构建多级 X 轴 ---
    x_prem = [df_plot[group_col].tolist(), ['保费'] * len(df_plot)]
    x_prof = [df_plot[group_col].tolist(), ['利润'] * len(df_plot)]
    
    # --- 6. 绘图 ---
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    colors = {'协议': '#aec7e8', '成数': '#98df8a', '溢额': '#ff9896', 'Open': '#c5b0d5', '临分': '#c49c94'}
    
    # === 柱状图 ===
    # 自留
    fig.add_trace(go.Bar(
        x=x_prem, y=df_plot['自留保费_万'],
        name='自留', legendgroup='group_ret', showlegend=True,
        marker_color='#1f77b4', hovertemplate='<b>%{x[0]}</b><br>自留保费: %{y:,.0f}万'
    ), secondary_y=False)
    
    fig.add_trace(go.Bar(
        x=x_prof, y=df_plot['自留利润_万'],
        name='自留', legendgroup='group_ret', showlegend=False,
        marker_color='#1f77b4', opacity=0.6, hovertemplate='<b>%{x[0]}</b><br>自留利润: %{y:,.0f}万'
    ), secondary_y=False)
    
    # 分出
    for name in reins_types.keys():
        fig.add_trace(go.Bar(
            x=x_prem, y=df_plot[f'{name}_保费_万'],
            name=name, legendgroup=f'group_{name}', showlegend=True,
            marker_color=colors[name], hovertemplate=f'<b>%{{x[0]}}</b><br>{name}保费: %{{y:,.0f}}万'
        ), secondary_y=False)
        
        fig.add_trace(go.Bar(
            x=x_prof, y=df_plot[f'{name}_利润_万'],
            name=name, legendgroup=f'group_{name}', showlegend=False,
            marker_color=colors[name], opacity=0.6, hovertemplate=f'<b>%{{x[0]}}</b><br>{name}利润: %{{y:,.0f}}万'
        ), secondary_y=False)

    # === 折线图 ===
    # 1. 自留COR
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['自留综合成本率'],
        name='自留综合成本率(%)', mode='lines+markers',
        line=dict(color='#d62728', width=3),
        hovertemplate='<b>%{x[0]}</b><br>自留COR: %{y:,.1f}%<br><i>(成本:%{customdata[0]:,.0f}万 / 保费:%{customdata[1]:,.0f}万)</i>',
        customdata=df_plot[['自留成本_万', '自留保费_万']]
    ), secondary_y=True)
    
    # 2. 毛COR
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['毛综合成本率'],
        name='毛综合成本率(%)', mode='lines',
        line=dict(color='#2ca02c', width=2),
        visible='legendonly'
    ), secondary_y=True)
    
    # 3. 分出总COR
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['总分出综合成本率'],
        name='分出总COR(%)', mode='lines+markers',
        line=dict(color='#9467bd', width=3)
    ), secondary_y=True)

    # 4. 分出比例
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['分出比例'],
        name='分出比例(%)', mode='lines',
        line=dict(color='black', dash='dot', width=1)
    ), secondary_y=True)
    
    # 5. 分项COR
    line_styles = {'协议': 'dot', '成数': 'dot', '溢额': 'dot', 'Open': 'dot', '临分': 'dash'}
    for name in reins_types.keys():
        if df_plot[f'{name}_COR'].sum() == 0: continue
        fig.add_trace(go.Scatter(
            x=x_prem, y=df_plot[f'{name}_COR'],
            name=f'{name}COR(%)', mode='lines',
            visible='legendonly',
            line=dict(color=colors[name], dash=line_styles[name], width=1.5)
        ), secondary_y=True)

    # --- 7. 布局优化 ---
    fig.update_layout(
        title=dict(text=f'<b>{group_col} 分出结构与双向效益分析 (2005年起)</b>', y=0.98, x=0.5, xanchor='center', yanchor='top'),
        height=800, 
        autosize=True,
        barmode='relative',
        hovermode="closest",
        legend=dict(orientation="h", y=1.28, x=0.5, xanchor="center", groupclick="toggleitem"),
        margin=dict(t=180, l=20, r=20, b=80), 
        xaxis=dict(title_text=""),
        
        # ⚡️ 恢复自动缩放 (移除 range 参数)
        yaxis2=dict(
            title_text="比率", 
            ticksuffix="%", 
            tickformat=".0f", 
            showgrid=False,
            # range=view_range  <-- 已移除，Plotly 将自动处理
            # fixedrange=False
        )
    )
    
    fig.add_hline(y=0, line_width=1, line_color="black")
    fig.update_yaxes(title_text="金额 (万元)", secondary_y=False)
    
    # --- 8. 表格 ---
    print(f"## 📊 {group_col} 指标明细 (单位: 万元/%)")
    cols = ['毛保费_万', '分出比例', '毛综合成本率', '自留综合成本率', '总分出综合成本率']
    display_df = df_plot[[group_col] + cols].round(1)
    
    rename_map = {
        '毛保费_万': '毛保费', 
        '毛综合成本率': '毛COR', 
        '自留综合成本率': '自留COR', 
        '总分出综合成本率': '分出总COR'
    }
    display_df = display_df.rename(columns=rename_map)
    
    print_markdown_table(pl.from_pandas(display_df))

    return fig

# 绑定并显示
dashboard.set_update_function(update_dashboard)
dashboard.show()

## 📊 业务年度 指标明细 (单位: 万元/%)


|   业务年度 |   毛保费 |   分出比例 |   毛COR |   自留COR |   分出总COR |
|-----------:|---------:|-----------:|--------:|----------:|------------:|
|       2005 |   4969.1 |       46.6 |    24.5 |       0.6 |        51.9 |
|       2006 |   9674.6 |       59.7 |   131.6 |      43.3 |       191.1 |
|       2007 |  14023.8 |       52.1 |    83.5 |      41.6 |       122.1 |
|       2008 |  26765   |       48.3 |    56.6 |      41.3 |        73   |
|       2009 |  38149.3 |       47.8 |    47.8 |      36.4 |        60.3 |
|       2010 |  52258.3 |       48.7 |    77.8 |      57.7 |        99   |
|       2011 |  53245.7 |       38.2 |    47.6 |      27.1 |        80.7 |
|       2012 |  62265.4 |       35.3 |    52.3 |      34.6 |        84.5 |
|       2013 |  66866.2 |       34.8 |    78   |      54.9 |       121.2 |
|       2014 |  76865.6 |       37.6 |    45.2 |      30.4 |        69.8 |
|       2015 |  69213.2 |       35.8 |    65.4 |      44.7 |       102.4 |
|       2016 |  90757.2 |       46.2 |    46.7 |      33.7 |        61.8 |
|       2017 |  77821.6 |       43.4 |    74.7 |      59.8 |        94.1 |
|       2018 |  87299.8 |       40.6 |    63.9 |      49   |        85.6 |
|       2019 |  91188.4 |       39.6 |    60.3 |      46   |        82.1 |
|       2020 |  85256.8 |       41.3 |    66.4 |      57.4 |        79.2 |
|       2021 |  94894.3 |       37.7 |    91.7 |      50.9 |       159.2 |
|       2022 |  94553   |       35   |    51.7 |      37.1 |        78.8 |
|       2023 |  97699.9 |       36.9 |    57.1 |      47.5 |        73.7 |
|       2024 |  99925.8 |       37   |    52.1 |      41.6 |        69.9 |
|       2025 |  80041.4 |       36.4 |    42   |      30.1 |        62.9 |

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] FlexBox(objects=[MultiChoice(name='📊 业务年度'...], sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

In [8]:
import panel as pn
import polars as pl
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import traceback
import time

pn.extension('plotly', sizing_mode='stretch_width')

# ==============================================================================
# 核心类：损益模拟器 (逻辑不变：瀑布扣减 + 变动XoL保费 + 无XoL赔付摊回)
# ==============================================================================
class ReinsuranceSimpleSim:
    def __init__(self, df_raw):
        try:
            max_year = df_raw['业务年度'].cast(pl.Int64).max()
        except:
            max_year = 2025
        start_year = 2005
        self.years = f"{start_year}-{max_year}"
        
        # 数据清洗
        self.df = df_raw.filter(
            (pl.col('业务年度').cast(pl.Int64) >= start_year) & 
            (pl.col('总保额') > 0)
        )
        
        # 准备基础账面数据
        cols_float = [
            '总保费', '总保额', '总已决赔款', '总未决赔款', '自留保费',
            '临分分保费', '临分摊回费用', '临分已决', '临分未决',
            '协议分保费', '溢额分保费', '协议摊回费用', '溢额摊回费用',
            '协议已决', '协议未决', '溢额已决', '溢额未决',
            '自留已决', '自留未决'
        ]
        
        self.df = self.df.with_columns([
            pl.col(c).cast(pl.Float64).fill_null(0.0) for c in cols_float
        ])
        
        self.df = self.df.with_columns([
            # Old Surplus
            (pl.col('协议分保费') + pl.col('溢额分保费')).alias('Old_Surplus_Prem'),
            (pl.col('协议摊回费用') + pl.col('溢额摊回费用')).alias('Old_Surplus_Comm'),
            (pl.col('协议已决') + pl.col('协议未决') + pl.col('溢额已决') + pl.col('溢额未决')).alias('Old_Surplus_Loss'),
            # Old Fac
            pl.col('临分分保费').alias('Old_Fac_Prem'),
            pl.col('临分摊回费用').alias('Old_Fac_Comm'),
            (pl.col('临分已决') + pl.col('临分未决')).alias('Old_Fac_Loss'),
            # Gross Loss
            (pl.col('总已决赔款') + pl.col('总未决赔款')).alias('Gross_Loss'),
            # Old Net Loss
            (pl.col('自留已决') + pl.col('自留未决')).alias('Old_Net_Loss')
        ])
        
        # Base Profit
        self.df = self.df.with_columns(
            (pl.col('自留保费') + pl.col('Old_Surplus_Comm') + pl.col('Old_Fac_Comm') - pl.col('Old_Net_Loss')).alias('Base_Profit')
        )

    def run_simulation(self, new_retention, xol_rate_pct):
        old_ret = 2.0e8
        
        # --- A. 瀑布扣减逻辑 ---
        df_sim = self.df.with_columns(
            (pl.min_horizontal(pl.col('总保额'), new_retention) - pl.min_horizontal(pl.col('总保额'), old_ret))
            .truediv(pl.col('总保额')).fill_nan(0.0).alias('Delta_Ratio')
        )
        
        df_sim = df_sim.with_columns([
            (pl.col('总保费') * pl.col('Delta_Ratio')).alias('Clawback_Prem'),
            (pl.col('Gross_Loss') * pl.col('Delta_Ratio')).alias('Clawback_Loss')
        ])
        
        # 1. 保费扣减 (先临分)
        df_sim = df_sim.with_columns([
            pl.max_horizontal(0.0, pl.col('Old_Fac_Prem') - pl.col('Clawback_Prem')).alias('Sim_Fac_Prem'),
            pl.max_horizontal(0.0, pl.col('Clawback_Prem') - pl.col('Old_Fac_Prem')).alias('Rest_Prem')
        ])
        df_sim = df_sim.with_columns([
            pl.max_horizontal(0.0, pl.col('Old_Surplus_Prem') - pl.col('Rest_Prem')).alias('Sim_Surplus_Prem')
        ])
        
        # 2. 赔款扣减 (先临分)
        df_sim = df_sim.with_columns([
            pl.max_horizontal(0.0, pl.col('Old_Fac_Loss') - pl.col('Clawback_Loss')).alias('Sim_Fac_Loss'),
            pl.max_horizontal(0.0, pl.col('Clawback_Loss') - pl.col('Old_Fac_Loss')).alias('Rest_Loss')
        ])
        df_sim = df_sim.with_columns([
            pl.max_horizontal(0.0, pl.col('Old_Surplus_Loss') - pl.col('Rest_Loss')).alias('Sim_Surplus_Loss')
        ])
        
        # 3. 手续费重算
        df_sim = df_sim.with_columns([
            pl.when(pl.col('Old_Fac_Prem') > 0)
              .then(pl.col('Old_Fac_Comm') * (pl.col('Sim_Fac_Prem') / pl.col('Old_Fac_Prem')))
              .otherwise(0.0).alias('Sim_Fac_Comm'),
            pl.when(pl.col('Old_Surplus_Prem') > 0)
              .then(pl.col('Old_Surplus_Comm') * (pl.col('Sim_Surplus_Prem') / pl.col('Old_Surplus_Prem')))
              .otherwise(0.0).alias('Sim_Surplus_Comm')
        ])
        
        # --- B. 自留重算 ---
        # Sim GNPI
        df_sim = df_sim.with_columns(
            (pl.col('总保费') - pl.col('Sim_Fac_Prem') - pl.col('Sim_Surplus_Prem')).alias('Sim_GNPI')
        )
        
        # ⚡️ Sim_Net_Loss = 毛赔款 - 模拟分出摊回 (不含XoL保护)
        df_sim = df_sim.with_columns(
            (pl.col('Gross_Loss') - pl.col('Sim_Fac_Loss') - pl.col('Sim_Surplus_Loss')).alias('Sim_Net_Loss')
        )

        # --- C. 年度聚合 ---
        yearly_agg = df_sim.group_by('业务年度').agg([
            pl.col('Base_Profit').sum(),
            pl.col('Sim_GNPI').sum(),
            (pl.col('总保费') - pl.col('Old_Fac_Prem') - pl.col('Old_Surplus_Prem')).sum().alias('Old_GNPI'),
            
            # 溢额合约
            pl.col('Old_Surplus_Prem').sum(),
            pl.col('Old_Surplus_Loss').sum(),
            pl.col('Old_Surplus_Comm').sum(),
            pl.col('Sim_Surplus_Prem').sum(),
            pl.col('Sim_Surplus_Loss').sum(),
            pl.col('Sim_Surplus_Comm').sum(),
            
            # 损益组件
            pl.col('Sim_Net_Loss').sum(),
            pl.col('Old_Net_Loss').sum(),
            pl.col('Old_Fac_Comm').sum(),
            pl.col('Sim_Fac_Comm').sum()
        ]).sort('业务年度')
        
        res = yearly_agg.to_pandas()
        for c in res.columns:
            if c != '业务年度': res[c] = pd.to_numeric(res[c], errors='coerce').fillna(0.0)
            
        # --- D. 衍生指标 ---
        
        res['Delta_GNPI'] = res['Sim_GNPI'] - res['Old_GNPI']
        res['Delta_Comm'] = (res['Sim_Surplus_Comm'] + res['Sim_Fac_Comm']) - (res['Old_Surplus_Comm'] + res['Old_Fac_Comm'])
        res['Delta_Loss'] = res['Sim_Net_Loss'] - res['Old_Net_Loss']
        
        # XoL 成本 (变动法)
        res['Delta_XoL_Cost'] = res['Delta_GNPI'] * (xol_rate_pct / 100.0)
        
        # 罚款
        def calc_penalty(row):
            prem = row['Sim_Surplus_Prem']
            loss = row['Sim_Surplus_Loss']
            if prem == 0: return 0.0
            lr = loss / prem
            if lr > 0.67:
                excess = loss - (prem * 0.67)
                cap = prem * 0.249
                return min(excess * 0.30, cap)
            return 0.0
            
        res['Cost_Penalty'] = res.apply(calc_penalty, axis=1)
        
        # 净利变化
        res['Delta_Total_Profit'] = res['Delta_GNPI'] + res['Delta_Comm'] - res['Delta_XoL_Cost'] - res['Cost_Penalty'] - res['Delta_Loss']
        
        res['Sim_Profit'] = res['Base_Profit'] + res['Delta_Total_Profit']
        
        # 合约绩效
        res['Treaty_Profit_Old'] = res['Old_Surplus_Prem'] - res['Old_Surplus_Loss'] - res['Old_Surplus_Comm']
        res['Treaty_Profit_Sim'] = res['Sim_Surplus_Prem'] - res['Sim_Surplus_Loss'] - res['Sim_Surplus_Comm']
        
        # 向量化计算利润率
        res['Treaty_Margin_Old'] = (res['Treaty_Profit_Old'] / res['Old_Surplus_Prem'] * 100).replace([np.inf, -np.inf, np.nan], 0.0).fillna(0.0)
        res['Treaty_Margin_Sim'] = (res['Treaty_Profit_Sim'] / res['Sim_Surplus_Prem'] * 100).replace([np.inf, -np.inf, np.nan], 0.0).fillna(0.0)
        
        return res

# ==============================================================================
# 交互界面 (更新选项)
# ==============================================================================

# ⚡️ 更新点：细化选项
ret_selector = pn.widgets.Select(
    name='模拟自留额', 
    options={
        '2.25亿': 2.25e8,
        '2.50亿': 2.50e8,
        '2.75亿': 2.75e8,
        '3.00亿': 3.00e8,
        '3.25亿': 3.25e8,
        '3.50亿': 3.50e8
    }, 
    value=3.0e8
)

xol_slider = pn.widgets.FloatSlider(name='XoL 费率 (GNPI %)', start=1.0, end=6.0, step=0.1, value=3.5)
run_btn = pn.widgets.Button(name='⚡ 执行细化测算', button_type='primary')

@pn.depends(run_btn.param.clicks)
def update_view(n_clicks):
    if n_clicks is None or n_clicks == 0:
        return pn.pane.Markdown("### 👈 请选择自留额并点击测算按钮")
    
    try:
        sim = ReinsuranceSimpleSim(df_df)
        df_res = sim.run_simulation(ret_selector.value, xol_slider.value)
        
        # --- 图表：损益归因堆叠图 ---
        fig = go.Figure()
        fig.add_trace(go.Bar(x=df_res['业务年度'], y=df_res['Delta_GNPI']/10000, name='保费增量(+)', marker_color='#2ecc71'))
        fig.add_trace(go.Bar(x=df_res['业务年度'], y=df_res['Delta_Comm']/10000, name='手续费变化(-)', marker_color='#f1c40f'))
        fig.add_trace(go.Bar(x=df_res['业务年度'], y=-df_res['Delta_XoL_Cost']/10000, name='新增XoL保费(-)', marker_color='#9b59b6'))
        fig.add_trace(go.Bar(x=df_res['业务年度'], y=-df_res['Cost_Penalty']/10000, name='罚款(-)', marker_color='#e74c3c'))
        fig.add_trace(go.Bar(x=df_res['业务年度'], y=-df_res['Delta_Loss']/10000, name='赔款增量(-)', marker_color='#e67e22'))
        
        fig.add_trace(go.Scatter(
            x=df_res['业务年度'], y=df_res['Delta_Total_Profit']/10000,
            name='净利变化(=)', mode='lines+markers',
            line=dict(color='black', width=3, dash='dot')
        ))
        
        fig.update_layout(
            title=f"损益变动归因 (模拟自留额: {ret_selector.value/1e8}亿)", 
            yaxis_title="变动金额 (万元)", barmode='relative', height=450
        )
        
        # --- 表格 ---
        tbl = df_res.copy()
        display_cols = [
            '业务年度', 
            'Delta_GNPI', 'Delta_Comm', 'Delta_XoL_Cost', 'Cost_Penalty', 'Delta_Loss',
            'Delta_Total_Profit',
            'Treaty_Margin_Old', 'Treaty_Margin_Sim'
        ]
        
        for c in display_cols:
            if 'Margin' not in c and c != '业务年度':
                tbl[c] = (tbl[c] / 10000).map('{:,.0f}'.format)
            elif 'Margin' in c:
                tbl[c] = tbl[c].map('{:.1f}%'.format)
        
        col_map = {
            '业务年度': '年度',
            'Delta_GNPI': '保费增量(+)',
            'Delta_Comm': '手续费变化(-)',
            'Delta_XoL_Cost': '新增XoL保费(-)',
            'Cost_Penalty': '罚款(-)',
            'Delta_Loss': '赔款增量(-)',
            'Delta_Total_Profit': '净利变化(=)',
            'Treaty_Margin_Old': '原合约利润率',
            'Treaty_Margin_Sim': '新合约利润率'
        }
        
        final_html = tbl[display_cols].rename(columns=col_map).to_html(classes=['table', 'table-striped', 'table-hover'], index=False)
        
        return pn.Column(
            pn.pane.Plotly(fig, sizing_mode='stretch_width'),
            pn.pane.Markdown("### 📋 损益变动明细 (不含XoL赔款摊回)"),
            pn.pane.HTML(f'<div style="overflow-x:auto">{final_html}</div>', sizing_mode='stretch_width')
        )
    except Exception as e:
        return pn.pane.Alert(f"❌ 运行错误: {str(e)}\n{traceback.format_exc()}", alert_type='danger')

layout = pn.Column(
    pn.pane.Markdown("## 🧮 自留额敏感性测试 (细化档位)"),
    pn.Row(ret_selector, xol_slider, run_btn),
    pn.layout.Divider(),
    update_view,
    sizing_mode='stretch_width'
)

layout.servable()
layout

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] Select(name='模拟自留额', options={'2.25亿': 225000000.0, ...}, sizing_mode='stretch_width', value=300000000.0)
        [1] FloatSlider(end=6.0, name='XoL 费率 (GNPI %)', sizing_mode='stretch_width', start=1.0, value=3.5)
        [2] Button(button_type='primary', name='⚡ 执行细化测算', sizing_mode='stretch_width')
    [2] Divider(sizing_mode='stretch_width')
    [3] ParamFunction(function, _pane=Markdown, defer_load=False, sizing_mode='stretch_width')

In [9]:
import importlib
import src.exporter
importlib.reload(src.exporter)
from src.exporter import ReportExporter
import os
from IPython.display import FileLink, display
target_obj = globals().get('dashboard')
if target_obj:
    report_path = ReportExporter.export_to_pdf(
        dashboard=target_obj,
        title="保险损益分析报告",
        author="分析师 Harold",
        filename="完美中文版"
    )
    
    if report_path:
        print("🎉 PDF 报告生成完成！")
        display(FileLink(report_path))

🚀 开始生成 PDF 报告: 完美中文版.pdf...
🔍 正在提取图表...
  🔄 尝试调用 update_function 生成图表...
## 📊 业务年度 指标明细 (单位: 万元/%)


|   业务年度 |   毛保费 |   分出比例 |   毛COR |   自留COR |   分出总COR |
|-----------:|---------:|-----------:|--------:|----------:|------------:|
|       2005 |   4969.1 |       46.6 |    24.5 |       0.6 |        51.9 |
|       2006 |   9674.6 |       59.7 |   131.6 |      43.3 |       191.1 |
|       2007 |  14023.8 |       52.1 |    83.5 |      41.6 |       122.1 |
|       2008 |  26765   |       48.3 |    56.6 |      41.3 |        73   |
|       2009 |  38149.3 |       47.8 |    47.8 |      36.4 |        60.3 |
|       2010 |  52258.3 |       48.7 |    77.8 |      57.7 |        99   |
|       2011 |  53245.7 |       38.2 |    47.6 |      27.1 |        80.7 |
|       2012 |  62265.4 |       35.3 |    52.3 |      34.6 |        84.5 |
|       2013 |  66866.2 |       34.8 |    78   |      54.9 |       121.2 |
|       2014 |  76865.6 |       37.6 |    45.2 |      30.4 |        69.8 |
|       2015 |  69213.2 |       35.8 |    65.4 |      44.7 |       102.4 |
|       2016 |  90757.2 |       46.2 |    46.7 |      33.7 |        61.8 |
|       2017 |  77821.6 |       43.4 |    74.7 |      59.8 |        94.1 |
|       2018 |  87299.8 |       40.6 |    63.9 |      49   |        85.6 |
|       2019 |  91188.4 |       39.6 |    60.3 |      46   |        82.1 |
|       2020 |  85256.8 |       41.3 |    66.4 |      57.4 |        79.2 |
|       2021 |  94894.3 |       37.7 |    91.7 |      50.9 |       159.2 |
|       2022 |  94553   |       35   |    51.7 |      37.1 |        78.8 |
|       2023 |  97699.9 |       36.9 |    57.1 |      47.5 |        73.7 |
|       2024 |  99925.8 |       37   |    52.1 |      41.6 |        69.9 |
|       2025 |  80041.4 |       36.4 |    42   |      30.1 |        62.9 |

  ✅ 通过调用 update_function 获取到 1 个图表
⚠️ 尝试加载 /System/Library/Fonts/Hiragino Sans GB.ttc 失败: TTC file "/System/Library/Fonts/Hiragino Sans GB.ttc": postscript outlines are not supported
✅ 成功加载中文字体: /System/Library/Fonts/STHeiti Medium.ttc

📊 开始处理 1 个图表...
  🖼️ 正在导出图表 1/1: <b>业务年度 分出结构与双向效益分析 (2005年起)</b>...
    ✅ 成功生成图片 (321.5 KB)
✅ 报告已成功导出至: outputs/reports/完美中文版.pdf
🎉 PDF 报告生成完成！


/Users/harold/working/Jupyter_AI_DataAnalyze/notebooks/02_analysis/outputs/reports/完美中文版.pdf

## 📤 Step 8: 导出和分享

如果 AI 代码中没有包含 `dashboard.save()`，手动导出：

In [10]:
from src.html_exporter import SimpleHTMLExporter
import os
from IPython.display import FileLink, display
import importlib
import src.html_exporter
importlib.reload(src.html_exporter)
from src.html_exporter import SimpleHTMLExporter
target_obj = globals().get('dashboard')
if target_obj:
    report_path = SimpleHTMLExporter.export_to_html(
        dashboard=target_obj,
        title="保险损益分析深度报告",
        author="分析师 Harold",
        filename="完整HTML版"
    )
    
    if report_path and os.path.exists(report_path):
        print("\n🎉 HTML 报告生成成功！")
        print("\n💡 使用方式:")
        print("   1. 点击下方链接在浏览器中打开")
        print("   2. 图表可以交互（缩放、悬停查看数据）")
        print("   3. 按 Cmd+P 可以打印或保存为 PDF")
        print()
        display(FileLink(report_path))

🚀 开始生成 HTML 报告: 完整HTML版.html...
🔍 正在提取图表...
  🔄 尝试调用 update_function 生成图表...
## 📊 业务年度 指标明细 (单位: 万元/%)


|   业务年度 |   毛保费 |   分出比例 |   毛COR |   自留COR |   分出总COR |
|-----------:|---------:|-----------:|--------:|----------:|------------:|
|       2005 |   4969.1 |       46.6 |    24.5 |       0.6 |        51.9 |
|       2006 |   9674.6 |       59.7 |   131.6 |      43.3 |       191.1 |
|       2007 |  14023.8 |       52.1 |    83.5 |      41.6 |       122.1 |
|       2008 |  26765   |       48.3 |    56.6 |      41.3 |        73   |
|       2009 |  38149.3 |       47.8 |    47.8 |      36.4 |        60.3 |
|       2010 |  52258.3 |       48.7 |    77.8 |      57.7 |        99   |
|       2011 |  53245.7 |       38.2 |    47.6 |      27.1 |        80.7 |
|       2012 |  62265.4 |       35.3 |    52.3 |      34.6 |        84.5 |
|       2013 |  66866.2 |       34.8 |    78   |      54.9 |       121.2 |
|       2014 |  76865.6 |       37.6 |    45.2 |      30.4 |        69.8 |
|       2015 |  69213.2 |       35.8 |    65.4 |      44.7 |       102.4 |
|       2016 |  90757.2 |       46.2 |    46.7 |      33.7 |        61.8 |
|       2017 |  77821.6 |       43.4 |    74.7 |      59.8 |        94.1 |
|       2018 |  87299.8 |       40.6 |    63.9 |      49   |        85.6 |
|       2019 |  91188.4 |       39.6 |    60.3 |      46   |        82.1 |
|       2020 |  85256.8 |       41.3 |    66.4 |      57.4 |        79.2 |
|       2021 |  94894.3 |       37.7 |    91.7 |      50.9 |       159.2 |
|       2022 |  94553   |       35   |    51.7 |      37.1 |        78.8 |
|       2023 |  97699.9 |       36.9 |    57.1 |      47.5 |        73.7 |
|       2024 |  99925.8 |       37   |    52.1 |      41.6 |        69.9 |
|       2025 |  80041.4 |       36.4 |    42   |      30.1 |        62.9 |

  ✅ 通过调用 update_function 获取到 1 个图表

📊 开始处理 1 个图表...
  ✅ 处理图表 1/1: <b>业务年度 分出结构与双向效益分析 (2005年起)</b>
✅ 报告已成功导出至: outputs/reports/完整HTML版.html
💡 提示: 打开 HTML 文件后，可以使用浏览器的 '打印' 功能保存为 PDF

🎉 HTML 报告生成成功！

💡 使用方式:
   1. 点击下方链接在浏览器中打开
   2. 图表可以交互（缩放、悬停查看数据）
   3. 按 Cmd+P 可以打印或保存为 PDF



/Users/harold/working/Jupyter_AI_DataAnalyze/notebooks/02_analysis/outputs/reports/完整HTML版.html

In [12]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import panel as pn

@pn.depends(*dashboard.widgets.values())
def update_dashboard(*args):
    # 获取控件值
    values = {name: widget.value for name, widget in dashboard.widgets.items()}
    
    # 获取聚合维度（关键步骤）
    group_col = values.get('_aggregation_dimension', '业务险种')
    
    # 构建过滤条件（跳过聚合维度控件）
    filters = []
    for dim, val in values.items():
        if dim == '_aggregation_dimension':
            continue
        
        if isinstance(val, list):
            if '全选' not in val:
                filters.append(pl.col(dim).is_in(val))
        else:
            if val != '全选':
                filters.append(pl.col(dim) == val)
    
    # 应用过滤条件
    filtered = df_df
    for f in filters:
        filtered = filtered.filter(f)
    
    # 确保只分析近5年数据
    current_year = datetime.now().year
    filtered = filtered.filter(
        (pl.col('业务年度').cast(pl.Int32) >= current_year - 5) &
        (pl.col('业务年度').cast(pl.Int32) <= current_year)
    )
    
    # 计算核心指标
    result = filtered.group_by(['业务年度', group_col]).agg([
        # 保费指标
        pl.col('总保费').sum().alias('总保费'),
        (pl.col('自留保费') + pl.col('附加自留保费')).sum().alias('毛自留保费'),
        ((pl.col('自留保费') + pl.col('附加自留保费')) - 
         (pl.col('分入手续费') + pl.col('分入出单费') + pl.col('分入经纪费'))).sum().alias('净自留保费'),
        
        # 赔款指标
        (pl.col('总已决赔款') + pl.col('总未决赔款')).sum().alias('总赔款'),
        (pl.col('自留已决') + pl.col('自留未决')).sum().alias('自留赔款'),
        (pl.col('附加自留已决') + pl.col('附加自留未决')).sum().alias('附加自留赔款'),
        
        # 成本指标
        (pl.col('分入手续费') + pl.col('分入出单费') + pl.col('分入经纪费')).sum().alias('分入成本'),
        (pl.col('手续费') + pl.col('分入手续费') + 
         pl.col('分入出单费') + pl.col('分入经纪费')).sum().alias('总获取成本'),
        
        # 分出方式指标
        pl.col('协议分保费').sum().alias('协议分出保费'),
        pl.col('成数分保费').sum().alias('成数分出保费'),
        pl.col('溢额分保费').sum().alias('溢额分出保费'),
        pl.col('临分分保费').sum().alias('临分分出保费'),
        
        # 分出赔款
        pl.col('协议已决').sum().alias('协议已决'),
        pl.col('协议未决').sum().alias('协议未决'),
        pl.col('成数已决').sum().alias('成数已决'),
        pl.col('成数未决').sum().alias('成数未决'),
        pl.col('溢额已决').sum().alias('溢额已决'),
        pl.col('溢额未决').sum().alias('溢额未决'),
        pl.col('临分已决').sum().alias('临分已决'),
        pl.col('临分未决').sum().alias('临分未决')
    ])
    
    # 计算衍生指标
    result = result.with_columns([
        # 比率指标
        (pl.col('总赔款') / pl.col('总保费')).alias('赔付率'),
        ((pl.col('总赔款') + pl.col('总获取成本')) / pl.col('总保费')).alias('综合成本率'),
        
        # 分出方式损益
        (pl.col('协议分出保费') - pl.col('协议已决') - pl.col('协议未决')).alias('协议分出损益'),
        (pl.col('成数分出保费') - pl.col('成数已决') - pl.col('成数未决')).alias('成数分出损益'),
        (pl.col('溢额分出保费') - pl.col('溢额已决') - pl.col('溢额未决')).alias('溢额分出损益'),
        (pl.col('临分分出保费') - pl.col('临分已决') - pl.col('临分未决')).alias('临分分出损益'),
        
        # 承保利润
        (pl.col('总保费') - pl.col('总赔款') - pl.col('总获取成本')).alias('承保利润')
    ])
    
    # 按业务年度和聚合维度排序
    result = result.sort(['业务年度', group_col])
    
    # 创建可视化图表
    if group_col == '业务年度':
        # 按年度分组时显示趋势图
        fig = go.Figure()
        
        # 添加保费指标
        fig.add_trace(go.Bar(
            x=result['业务年度'],
            y=result['总保费'],
            name='总保费',
            marker_color='#636EFA'
        ))
        fig.add_trace(go.Bar(
            x=result['业务年度'],
            y=result['毛自留保费'],
            name='毛自留保费',
            marker_color='#EF553B'
        ))
        fig.add_trace(go.Bar(
            x=result['业务年度'],
            y=result['净自留保费'],
            name='净自留保费',
            marker_color='#00CC96'
        ))
        
        # 添加比率指标（次坐标轴）
        fig.add_trace(go.Scatter(
            x=result['业务年度'],
            y=result['赔付率'],
            name='赔付率',
            yaxis='y2',
            mode='lines+markers',
            line=dict(color='#19D3F3', width=2)
        ))
        fig.add_trace(go.Scatter(
            x=result['业务年度'],
            y=result['综合成本率'],
            name='综合成本率',
            yaxis='y2',
            mode='lines+markers',
            line=dict(color='#FF6692', width=2)
        ))
        
        fig.update_layout(
            title=f'近五年保险业务损益分析（按{group_col}）',
            barmode='group',
            xaxis_title='业务年度',
            yaxis_title='金额（元）',
            yaxis2=dict(
                title='比率',
                overlaying='y',
                side='right',
                range=[0, 1.5]
            ),
            height=600
        )
    else:
        # 按其他维度分组时显示柱状图
        fig = px.bar(
            result.to_pandas(),
            x=group_col,
            y='总保费',
            color='业务年度',
            barmode='group',
            title=f'近五年保险业务分析（按{group_col}）',
            labels={'总保费': '总保费（元）'}
        )
        fig.update_layout(height=600)
    
    return fig

# 绑定更新函数
dashboard.set_update_function(update_dashboard)

# 显示仪表盘
dashboard.show()


Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] FlexBox(objects=[MultiChoice(name='📊 业务年度'...], sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')